In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
api_key=os.getenv("GROQ_API_KEY")

model="deepseek-r1-distill-llama-70b"
api_key


''

In [17]:
from langchain_groq import ChatGroq
llm=ChatGroq( model_name=model,api_key=api_key )
llm.invoke("hi")

APIConnectionError: Connection error.